In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


### Build dataset and trim features

In [2]:
# all of our usable dataframes
df = pd.read_csv('data/master_df.csv')
nlp_data = df[['blurb', 'slug', 'country']]  #also includes slug from category and some info from location

In [3]:
# read category data for name and parent name
df_category = pd.read_csv('data/category.csv')
df_category = df_category[['name', 'parent_name']]

In [4]:
# drop columns deemed unessecary
model_data = df.drop(columns = ['is_starrable','current_currency','static_usd_rate','usd_exchange_rate','usd_type','id','name', 'slug', 'category', 'creator', 'location', 'photo', 'profile', 'urls', 'country_displayable_name', 'currency_symbol', 'currency_trailing_code', 'disable_communication', 'source_url', 'currency', 'pledged', 'blurb'])

# create goal_usd column so that all goal amounts are in the same units, drop fx_rate and goals afterward
model_data['goal_usd'] = model_data['fx_rate']*model_data['goal']
model_data = model_data.drop(columns=['fx_rate', 'goal'])

# combine category data and all other date
model_data = pd.concat([model_data, df_category], axis=1, join='inner')

# create percentage funded column as another metric
model_data['percentage_funded'] = model_data.converted_pledged_amount/model_data.goal_usd*100

# create total days active column as another metric
model_data['total_days_active'] = (model_data.deadline-model_data.created_at)*0.00001157

model_data

,backers_count,converted_pledged_amount,country,created_at,deadline,launched_at,spotlight,staff_pick,state,state_changed_at,usd_pledged,goal_usd,name,parent_name,percentage_funded,total_days_active
0,18,5034,US,1609376406,1611968831,1609545583,True,False,successful,1611968831,5034.000000,5000.000000,Cookbooks,Food,100.680000,29.994357
1,508,48365,HK,1606278560,1614096106,1608912106,True,True,successful,1614096106,48368.291331,44624.926500,Cookbooks,Food,108.381131,90.449007
2,14,98,ES,1606477096,1607900340,1606751262,True,False,successful,1607900340,96.906412,54.095050,Cookbooks,Food,181.162603,16.466933
3,1486,127765,US,1604500905,1610082068,1606194068,True,True,successful,1610082068,127765.690000,65000.000000,Cookbooks,Food,196.561538,64.574056
4,249,14574,GB,1605454727,1608573895,1605981895,True,True,successful,1608573895,14480.304767,13823.881138,Cookbooks,Food,105.426254,36.088774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30987,4,501,US,1632678330,1634318870,1633022870,False,False,failed,1634318870,501.000000,10000.000000,Software,Technology,5.010000,18.981048
30988,1,1,CA,1632726042,1638144718,1632957118,False,False,failed,1638144719,1.022898,39722.546000,Software,Technology,0.002517,62.694081
30989,1,0,DK,1631517750,1635408514,1632816514,False,False,failed,1635408514,0.972658,145430.220000,Software,Technology,0.000000,45.016139
30990,3,70,US,1429554526,1432913659,1430321659,False,False,failed,1432913660,70.000000,35000.000000,Plays,Theater,0.200000,38.865169


In [5]:
model_data.isna().sum().sum()

1374

In [6]:
# drop rows with na values
model_data = model_data.dropna()

# drop rows that have state 'live' or 'cancelled'
model_data = model_data.drop(model_data[model_data['state'] == 'live'].index, axis=0)
model_data = model_data.drop(model_data[model_data['state'] == 'canceled'].index, axis=0)
model_data = model_data.reset_index(drop=True)
model_data.head()

,backers_count,converted_pledged_amount,country,created_at,deadline,launched_at,spotlight,staff_pick,state,state_changed_at,usd_pledged,goal_usd,name,parent_name,percentage_funded,total_days_active
0,18,5034,US,1609376406,1611968831,1609545583,True,False,successful,1611968831,5034.000000,5000.000000,Cookbooks,Food,100.680000,29.994357
1,508,48365,HK,1606278560,1614096106,1608912106,True,True,successful,1614096106,48368.291331,44624.926500,Cookbooks,Food,108.381131,90.449007
2,14,98,ES,1606477096,1607900340,1606751262,True,False,successful,1607900340,96.906412,54.095050,Cookbooks,Food,181.162603,16.466933
3,1486,127765,US,1604500905,1610082068,1606194068,True,True,successful,1610082068,127765.690000,65000.000000,Cookbooks,Food,196.561538,64.574056
4,249,14574,GB,1605454727,1608573895,1605981895,True,True,successful,1608573895,14480.304767,13823.881138,Cookbooks,Food,105.426254,36.088774


In [7]:
# split into X and y variables
X = model_data.drop(columns=['state'])
y = model_data.state.to_frame()

In [8]:
# generate 70% train/test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

### Encoding of non neumerical features

In [9]:
# init encoder for x values
encoder = LabelEncoder()

# encode boolean and string type columns
encoding_columns = (X.select_dtypes(include=['bool', 'object'])).columns
for column in encoding_columns:
    X_train[column] = encoder.fit_transform(X_train[column])
    X_test[column] = encoder.transform(X_test[column])

In [10]:
# new encoder for y variable (to make sure that we can reverse encoding)
y_encoder = LabelEncoder()
y_train['state'] = y_encoder.fit_transform(y_train['state'])
y_test['state'] = y_encoder.transform(y_test['state'])

### Scale values using standard scaler

In [11]:
# init standard scaler
scaler = MinMaxScaler()

# scale Train and testing set, columns=X.columns to keep column headers.
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
X_train_scaled.head()

,backers_count,converted_pledged_amount,country,created_at,deadline,launched_at,spotlight,staff_pick,state_changed_at,usd_pledged,goal_usd,name,parent_name,percentage_funded,total_days_active
0,0.000054,1.250175e-05,1.0,0.704189,0.703488,0.703632,0.0,0.0,0.703488,1.250175e-05,0.000012,0.578947,0.666667,6.325985e-06,0.006430
1,0.000286,4.406747e-05,1.0,0.494477,0.494492,0.494772,1.0,0.0,0.494492,4.406747e-05,0.000014,0.289474,0.777778,1.845392e-05,0.008988
2,0.000162,4.765993e-05,1.0,0.964227,0.976434,0.973497,1.0,1.0,0.976434,4.765993e-05,0.000010,0.368421,0.111111,2.893956e-05,0.021282
3,0.000011,9.579885e-08,1.0,0.972951,0.980761,0.977795,0.0,0.0,0.980761,9.579885e-08,0.000800,0.131579,0.444444,7.271247e-10,0.015471
4,0.000658,4.221137e-04,1.0,0.969724,0.986289,0.983286,1.0,0.0,0.986289,4.221137e-04,0.000150,0.763158,0.444444,1.708743e-05,0.026931


### Build Random forest classifier

In [12]:
# init and fit RF classifier
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_scaled, y_train.state)

RandomForestClassifier()

In [13]:
# perform predictions on testing data
predictions  = rf_classifier.predict(X_test_scaled)

In [14]:
# print classification report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2895
           1       1.00      1.00      1.00      5483

    accuracy                           1.00      8378
   macro avg       1.00      1.00      1.00      8378
weighted avg       1.00      1.00      1.00      8378



In [15]:
# view confusion matrix
cm = confusion_matrix(y_test, predictions)
cm

array([[2895,    0],
       [   0, 5483]])